# Netflix Case Study: 
## The Effect of TV Shows and Movies on the Stock Price.
<img src="https://images.ctfassets.net/y2ske730sjqp/5QQ9SVIdc1tmkqrtFnG9U1/de758bba0f65dcc1c6bc1f31f161003d/BrandAssets_Logos_02-NSymbol.jpg?w=940" alt="Netflix Logo" width="300" height="200">

### Table of Contents
[1. Import Libraries and Data](#import)

[2. Genre](#genre)

<!-- import -->
### Import the Libraries 

In [1]:
siondoiasnd

NameError: name 'siondoiasnd' is not defined

<!--genre-->
### Genre